In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from scipy.special import inv_boxcox

In [2]:
df = pd.read_csv('data/NBA_Player_Dataset-2013-2021.csv')

box_cox_transformer = 0.16106145323461019 #need to pull from data_prep.ipynb

In [3]:
#Can be used to filter out earlier years
#df = df[~df['Year'].isin([2013, 2014, 2015])]

In [4]:
df.columns

Index(['Year', 'FULL NAME', 'TEAM', 'POS', 'AGE', 'GP', 'MPG', 'MIN%', 'USG%',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'VI', 'ORTG', 'DRTG',
       'TO_100_Games', 'Salary', 'Cap Maximum', 'Salary%OfCap', 'Traded',
       'Center', 'Forward', 'Guard', 'Salary_BoxCox'],
      dtype='object')

In [5]:
#variables we need to scale

x_to_scale = df[['Year', 'AGE', 'GP', 'MPG', 'MIN%', 'USG%',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'VI', 'TO_100_Games']]

In [6]:
'''x_to_scale = df[['Year', 'AGE', 'GP', 'MPG', 'PPG', 'RPG',
       'APG', 'SPG', 'BPG']]'''

"x_to_scale = df[['Year', 'AGE', 'GP', 'MPG', 'PPG', 'RPG',\n       'APG', 'SPG', 'BPG']]"

In [7]:
#scaler function
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_to_scale)

x_scaled = scaler.transform(x_to_scale)

x_scaled_df = pd.DataFrame(x_scaled, columns = x_to_scale.columns)

In [8]:
x_scaled_df

,Year,AGE,GP,MPG,MIN%,USG%,FTA,FT%,2PA,2P%,...,TS%,PPG,RPG,TRB%,APG,AST%,SPG,BPG,VI,TO_100_Games
0,1.482654,-0.955940,0.448998,0.349759,0.352046,-0.094111,-0.015981,-0.539477,0.411389,-0.273432,...,-0.372734,-0.015990,1.275664,1.115718,-0.442791,-0.660957,-0.245491,0.399570,0.122456,-0.192193
1,1.482654,0.502305,0.693373,0.695611,0.704788,-1.107608,0.602635,-0.746889,0.266076,0.568375,...,0.373600,-0.227872,2.530513,1.792715,0.775668,0.273075,0.443012,0.853779,1.189651,1.127463
2,1.482654,-0.440709,-0.161940,1.408931,1.410274,1.164023,1.355006,0.237111,1.252678,0.458232,...,0.585522,1.695360,2.692429,1.444545,0.886437,0.533735,2.081174,0.949402,1.744592,0.290608
3,1.482654,2.393059,-0.161940,0.306527,0.315734,0.709697,-0.041060,0.825582,0.533759,0.733590,...,0.723732,0.798938,0.789916,0.612805,-0.553560,-0.726122,-0.767804,1.475328,0.250520,-0.819835
4,1.482654,-0.707778,0.367540,-0.314926,-0.306753,0.456323,0.000739,0.526523,0.239308,-0.812346,...,-0.114742,-0.105633,-0.505412,-0.276964,-0.055099,0.159036,-0.233621,-0.353460,0.463959,-0.224380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4205,-1.606858,-1.544437,0.937748,0.468645,0.450606,1.618350,1.831507,-0.293477,1.336807,0.041262,...,-0.326664,0.717445,-0.181580,-0.605791,0.609514,0.881282,1.060290,-0.437130,0.677398,0.773409
4206,-1.606858,0.346317,0.611915,0.879345,0.870785,1.461083,1.488760,0.594053,1.027060,-0.147555,...,0.327530,1.516076,-0.424454,-1.011990,-0.221253,-0.432879,0.229338,-0.508847,-0.304422,-0.691088
4207,-1.606858,-0.362716,1.222852,1.538626,1.508834,0.989283,1.020619,0.048993,2.755526,0.072731,...,-0.151598,1.516076,0.951832,-0.122221,0.221823,-0.128776,3.553146,0.136608,0.293207,-0.353127
4208,-1.606858,-1.308093,1.345040,-0.309522,-0.327502,0.080630,0.778188,0.135817,0.506990,-0.470116,...,-0.280594,-0.423455,0.263689,0.786890,-0.442791,-0.280828,-0.340457,0.232230,0.207832,0.338888


In [9]:
#add the categorical features to scaled features for input
X = pd.concat([x_scaled_df, df[['Center', 'Forward', 'Guard']]], axis = 1)

#choose our target variable
#y = df['Salary']
y = df['Salary_BoxCox']

# Feature Selection
SelectKBest
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

### Scorers for regression
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif

In [10]:
#Need to do

# Grid Search CV

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

https://scikit-learn.org/stable/modules/grid_search.html

# Random Forest

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

#call the model you want to use, define any parameters you want to hold steady across the search
random_forest = RandomForestRegressor(verbose = 1)

#Create a search grid of parameters
rf_param_grid = [
  {'n_estimators': [100, 250], 'max_depth': [10, 25], 'max_features': [5, 10, 20]},
 ]


#Create grid search, the model, the grid, scoring metric, 
#verbose just makes it print output while training,
#njobs = -1 runs on all cores of your pc when training.  
rf_gridsearch = GridSearchCV(random_forest, rf_param_grid, 
                scoring = 'neg_root_mean_squared_error', 
                verbose = 2, n_jobs = -1)

#grid search maximizes, so scorer needs to be negative in this case.  Real MSE is just the positive version

In [14]:
rf_gridsearch.fit(X, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.6s finished


GridSearchCV(estimator=RandomForestRegressor(verbose=1), n_jobs=-1,
             param_grid=[{'max_depth': [10, 25], 'max_features': [5, 10, 20],
                          'n_estimators': [100, 250]}],
             scoring='neg_root_mean_squared_error', verbose=2)

In [21]:
random_forest_results = pd.DataFrame(rf_gridsearch.cv_results_)

#replace -mse with +mse
random_forest_results[['split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score']] = random_forest_results[['split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score']] * -1.0

#add label to pull merge data later
random_forest_results['Model Type'] = 'Random Forest'

In [ ]:
#write to csv to save results
random_forest_results.to_csv('results/rf_results_all_vars.csv', index = False)

In [23]:
random_forest_results.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,Model Type
9,7.292389,0.396307,0.123504,0.033118,25,10,250,"{'max_depth': 25, 'max_features': 10, 'n_estim...",8.702257,9.551042,10.750902,8.306658,8.896781,9.241528,0.855296,1,Random Forest
3,4.691824,0.316998,0.080200,0.007417,10,10,250,"{'max_depth': 10, 'max_features': 10, 'n_estim...",8.728089,9.500413,10.796469,8.304252,8.976986,9.261242,0.859652,2,Random Forest
11,10.011807,0.465994,0.077395,0.004592,25,20,250,"{'max_depth': 25, 'max_features': 20, 'n_estim...",8.764966,9.553596,10.805070,8.326632,8.874918,9.265036,0.864620,3,Random Forest
8,2.581800,0.063644,0.046203,0.004708,25,10,100,"{'max_depth': 25, 'max_features': 10, 'n_estim...",8.733716,9.542834,10.771338,8.342741,8.981572,9.274440,0.843677,4,Random Forest
5,8.681680,0.433435,0.083509,0.010403,10,20,250,"{'max_depth': 10, 'max_features': 20, 'n_estim...",8.740017,9.571033,10.858909,8.309480,8.912417,9.278371,0.888397,5,Random Forest
4,3.361783,0.245649,0.036201,0.003762,10,20,100,"{'max_depth': 10, 'max_features': 20, 'n_estim...",8.762839,9.558227,10.907710,8.364728,8.921707,9.303042,0.889691,6,Random Forest
10,4.849396,0.465983,0.042599,0.004759,25,20,100,"{'max_depth': 25, 'max_features': 20, 'n_estim...",8.777524,9.605465,10.830744,8.383033,8.924857,9.304324,0.859178,7,Random Forest
7,3.994942,0.181543,0.118199,0.009260,25,5,250,"{'max_depth': 25, 'max_features': 5, 'n_estima...",8.723643,9.531604,10.873404,8.385665,9.015635,9.305990,0.869266,8,Random Forest
2,1.928196,0.098584,0.037226,0.003673,10,10,100,"{'max_depth': 10, 'max_features': 10, 'n_estim...",8.846254,9.581709,10.893787,8.340244,8.970295,9.326458,0.877695,9,Random Forest
6,1.579833,0.060390,0.044799,0.004445,25,5,100,"{'max_depth': 25, 'max_features': 5, 'n_estima...",8.834120,9.581106,10.807104,8.441396,8.969706,9.326687,0.825853,10,Random Forest


# AdaBoost

# Notes

Once we come up with best performing model, can grab the best estimator from all the training.

Train the model on all previous season data.  Apply a scaling factor where appropriate on current season data, then predict on it.  

Inaccuracies aren't exactly *wrong*.  We can consider the results what a player is actually worth.  Doing some testing on the simple models, Steph Curry in 2019 was worst prediction.  Something like 30M difference.  But that season he was hurt and only played 5 games.  So maybe he wasn't worth his 40M contract?  

Anyway we can see what our model thinks a player is worth, highlight some highs and lows.  Then do a groupby on teams and do the same.  